In [ ]:
!apt-get update
!apt install -y firefox
!wget https://github.com/mozilla/geckodriver/releases/download/v0.34.0/geckodriver-v0.34.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.34.0-linux64.tar.gz
!mv geckodriver /usr/local/bin/
!pip install selenium
!pip install geckodriver
!pip install python_dotenv
!pip install mysql-connector-python
!pip install slack_sdk

import os
import mysql.connector
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import re
import numpy as np
import pandas as pd
import uuid

In [ ]:
#Extracting sanrafael data

class SanRafaelDataExtractor:
    def __init__(self):
        self.options = webdriver.FirefoxOptions()
        self.options.add_argument('--headless')
        self.driver = webdriver.Firefox(options=self.options)
        load_dotenv('/root/Desktop/infra/cred.env')
        db_host=os.environ.get('DB_HOST')
        db_user=os.environ.get('DB_USER')
        db_password=os.environ.get('DB_PASSWORD')
        self.sanrafael_url=os.environ.get('SANRAFAEL_URL')
        self.mydb=mysql.connector.connect(
            host=db_host,
            user=db_user,
            password=db_password
        )

    def extract_data(self):
        # Get the webpage
        self.driver.get(self.sanrafael_url)
        self.driver.execute_script("window.scrollBy(0, 500);")
        sanrafael_table = self.driver.find_element('css selector', 'tbody')
        sanrafael_rows = sanrafael_table.find_elements('css selector', "tr")
        data = []
        for row in sanrafael_rows[1:]:
            cells = row.find_elements('css selector', "td")
            row_data = [cell.text for cell in cells]
            data.append(row_data)
        return data

    def save_to_csv(self, data, headers):
        with open('sanrafael.csv', 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(headers)
            csv_writer.writerows(data)

    def cleanup_data(self, data):
        #Create an empty dataframe with the desired columns
        sanrafael_df = pd.DataFrame(columns=['original_id','aug_id','country_name','country_code','map_coordinates','url','region_name','region_code','title', 'description', 'status','stages','date','procurement_method','budget','currency','buyer','sector','subsector','location'])
        sanrafael_csv = pd.DataFrame(data, columns=['title', 'description', 'floor area', 'Number Units', 'BMR Units', 'applicant', 'staff', 'status'])
        sanrafael_csv.drop(['floor area', 'Number Units', 'BMR Units', 'applicant', 'staff'], axis=1, inplace=True)
        #assigning uuid for each row
        for _ in range(len(sanrafael_csv)):
            sanrafael_df['aug_id']=[str(uuid.uuid4()) for _ in range(len(sanrafael_csv))]
            sanrafael_df['url']=[self.sanrafael_url for _ in range(len(sanrafael_csv))]
        columns = ['title','description','status']
        for column in columns:
            sanrafael_df[column]=sanrafael_csv[column]
        return sanrafael_df

    def close_driver(self):
        self.driver.quit()

    def insert_data(self):
        mycursor = self.mydb.cursor()
        with open('modified_sanrafael.csv') as file:
            rows=csv.reader(file)
            next(rows)
            val=[tuple(row) for row in rows]
        query="INSERT INTO sanrafael_source (original_id,aug_id,country_name,country_code,map_coordinates,url,region_name,region_code,title,description,status,stages,published_date,procurement_method,budget,currency,buyer,sector,subsector,location) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute("USE infraDB")
        mycursor.executemany(query,val)
        self.mydb.commit()

    def send_alerts(self):
        slack_token=os.environ.get('SLACK_BOT_TOKEN')
        #intialize webclient instance with OAtuh token
        client=WebClient(token=slack_token)
        #fetch the updated records from the cdc
        sanrafael_cdc=pd.read_sql("SELECT*FROM sanrafael_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
        #create a csv filename which accumulates updated cdc records 
        filename="sanrafael_projects"+"_"+str(datetime.now())+".csv"
        sanrafael_cdc.to_csv(filename,index=False)
        #counting the number of projects
        new_projects=pd.read_sql("SELECT COUNT(*) AS total_projects FROM sanrafael_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
        project_count=str(int(new_projects.iloc[0]))
        #customization of notification that has to be sent to slack channel
        notification= "Update:"+" "+project_count+" "+"infrastructure projects indentified in San Rafael"
        try:
            client.chat_postMessage(channel="california_infra_projects",text=notification,username="infra_alerts")
            client.files_upload_v2(channel="C07AH45FZDZ",file=filename)
        except SlackApiError as e:
            print(f"Error in sending message:{e}")
            

if __name__ == "__main__":
    headers = ['title','description','floor area','Number Units','BMR Units','applicant','staff','status']
    sanrafael_extractor = SanRafaelDataExtractor()
    data = sanrafael_extractor.extract_data()
    sanrafael_extractor.save_to_csv(data, headers)
    modified_data = sanrafael_extractor.cleanup_data(data)
    modified_data.to_csv('modified_sanrafael.csv', index=False, na_rep='NULL')
    sanrafael_extractor.close_driver()
    sanrafael_extractor.insert_data()
    sanrafael_extractor.send_alerts()

In [ ]:
# Extracting Fairfield data

class FairFieldDataExtractor:
  def __init__(self):
    self.options=webdriver.FirefoxOptions()
    self.options.add_argument("--headless")
    self.driver = webdriver.Firefox(options=self.options)
    load_dotenv('/root/Desktop/infra/cred.env')
    db_host=os.environ.get('DB_HOST')
    db_user=os.environ.get('DB_USER')
    db_password=os.environ.get('DB_PASSWORD')
    self.fairfield_url=os.environ.get('FAIRFIELD_URL')
    self.mydb=mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password
        )

  def extract_data(self):
    # Get the webpage
    self.driver.get(self.fairfield_url)
    # Close the pop-up
    self.driver.find_element('css selector', '.prefix-overlay-close.prefix-overlay-action-later').click()
    # Scroll down by 500 pixels
    self.driver.execute_script("window.scrollBy(0,500)")
    # Find the table using CSS Selector
    fairfield_table = self.driver.find_element('css selector', "tbody")
    # Get all the rows from the table using find element
    fairfield_rows = fairfield_table.find_elements('css selector', "tr")
    fair_data=[]
    # Iterate through the rows and extract data
    for row in fairfield_rows:
      # Get all cells present in the current row
      cells = row.find_elements('css selector', "td")
      # Extract and write data from each cell to csv file
      row_data = [cell.text for cell in cells]
      fair_data.append(row_data)
    return fair_data

  def save_to_csv(self,data,headers):
    # Create a csv file to store results
    with open('fairfield.csv', 'w', newline='') as csvfile:
      csv_writer = csv.writer(csvfile)
      csv_writer.writerow(headers)
      csv_writer.writerows(data)

  def cleanup_data(self,data):
    #Create an empty dataframe with the desired columns
    fairfield_df = pd.DataFrame(columns=['original_id','aug_id','country_name','country_code','map_coordinates','url','region_name','region_code','title', 'description', 'status','stages','date','procurementMethod','budget','currency','buyer','sector','subsector'])
    #read the csv file to dataframe
    fairfield_csv = pd.read_csv('fairfield.csv')
    #drop the unnecessary rows
    fairfield_csv.drop([0,1],axis=0,inplace=True)
   #assigning uuid and url for each row
    for _ in range(len(fairfield_csv)):
        fairfield_df['aug_id']=[str(uuid.uuid4()) for _ in range(len(fairfield_csv))]
        fairfield_df['url']=[self.fairfield_url for _ in range(len(fairfield_csv))]
    #mapping columns
    columns = ['original_id','title','location','subsector']
    for column in columns:
      fairfield_df[column]=fairfield_csv[column]
    return fairfield_df
      
  def insert_data(self):
      mycursor = self.mydb.cursor()
      with open('modified_fairfield.csv') as file:
          rows=csv.reader(file)
          next(rows)
          val=[tuple(row) for row in rows]
      query="INSERT INTO fairfield_source (original_id,aug_id,country_name,country_code,map_coordinates,url,region_name,region_code,title,description,status,stages,published_date,procurement_method,budget,currency,buyer,sector,subsector,location)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
      mycursor.execute("USE infraDB")
      mycursor.executemany(query,val)
      self.mydb.commit()

  def send_alerts(self):
      slack_token=os.environ.get('SLACK_BOT_TOKEN')
      #intialize webclient instance with OAtuh token
      client=WebClient(token=slack_token)
      #fetch the updated records from the cdc
      fairfield_cdc=pd.read_sql("SELECT*FROM fairfield_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
      #create a csv filename which accumulates updated cdc records 
      filename="fairfield_projects"+"_"+str(datetime.now())+".csv"
      fairfield_cdc.to_csv(filename,index=False)
      #counting the number of projects
      total_projects=pd.read_sql("SELECT COUNT(*) AS total_projects FROM fairfield_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
      project_count=str(int(total_projects.iloc[0]))
      #customization of notification that has to be sent to slack channel
      notification="Update :"+" "+project_count+" "+"infrastructure projects indentified in Fairfield"
      try:
          client.chat_postMessage(channel="california_infra_projects",text=notification,username="infra_alerts")
          client.files_upload_v2(channel="C07AH45FZDZ",file=filename)
      except SlackApiError as e:
          print(f"Error in sending message:{e}")

  def close_driver(self):
    self.driver.quit()


if __name__ == "__main__":
    headers = ['original_id','title','location','subsector']
    fairfield_extractor = FairFieldDataExtractor()
    fairfield_data = fairfield_extractor.extract_data()
    fairfield_extractor.save_to_csv(fairfield_data, headers)
    fairfield_modified_data = fairfield_extractor.cleanup_data(fairfield_data)
    fairfield_modified_data.to_csv('modified_fairfield.csv', index=False, na_rep='NULL')
    fairfield_extractor.close_driver()
    fairfield_extractor.insert_data()
    fairfield_extractor.send_alerts()

In [ ]:
#extracting elk
class ElkGroveDataExtractor:
  def __init__(self):
    self.options= webdriver.FirefoxOptions()
    self.options.add_argument("--headless")
    self.driver= webdriver.Firefox(options =self.options)
    self.elk_url=os.environ.get('ELK_URL')
    load_dotenv('/root/Desktop/infra/cred.env')
    db_host=os.environ.get('DB_HOST')
    db_user=os.environ.get('DB_USER')
    db_password=os.environ.get('DB_PASSWORD')
    self.mydb=mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password
    )

  def extract_data(self):
    # Get the webpage
    self.driver.get(self.elk_url)
    #Get all rows
    elk_rows=self.driver.find_elements('css selector',"tr")
    elk_data = []
    for row in elk_rows:
      # Get all the cells present in the row
      cells = row.find_elements('css selector', "td")
      # Extract each cell and write the data to csv
      row_data = [cell.text for cell in cells]
      elk_data.append(row_data)
    return elk_data

  def save_to_csv(self,data,headers):
    with open('elk.csv', 'w', newline='') as csvfile:
       csv_writer = csv.writer(csvfile)
       self.driver.execute_script("window.scrollBy(0,550)")
       csv_writer.writerow(headers)
       csv_writer.writerows(data)

  def cleanup_data(self,data):
    # Create an empty dataframe with the desired columns
    elk_df = pd.DataFrame(columns=['original_id','aug_id','country_name','country_code','map_coordinates','url','region_name','region_code','title', 'description', 'status','stages','date','procurement_method','budget','currency','buyer','sector','subsector','location'])
    #read the csv file to dataframe
    elk_csv = pd.read_csv('elk.csv')
    #assigning uuid for each row
    for _ in range(len(elk_csv)):
        elk_df['aug_id'] = [str(uuid.uuid4()) for _ in range(len(elk_csv))]
        elk_df['url'] = [self.elk_url for _ in range(len(elk_csv))]
    #mapping columns
    mapping_columns = ['title','description','status']
    for column in mapping_columns:
      elk_df[column] = elk_csv[column]
    return elk_df

  def insert_data(self):
        mycursor = self.mydb.cursor()
        with open('modified_elk.csv') as file:
            rows=csv.reader(file)
            next(rows)
            val=[tuple(row) for row in rows]
        query="INSERT INTO elk_source (original_id,aug_id,country_name,country_code,map_coordinates,url,region_name,region_code,title,description,status,stages,published_date,procurement_method,budget,currency,buyer,sector,subsector,location)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute("USE infraDB")
        mycursor.executemany(query,val)
        self.mydb.commit()
      
  def send_alerts(self):
      slack_token=os.environ.get('SLACK_BOT_TOKEN')
      #intialize webclient instance with OAtuh token
      client=WebClient(token=slack_token)
      #fetch the updated records from the cdc
      elk_cdc=pd.read_sql("SELECT*FROM elk_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
      #create a csv filename which accumulates updated cdc records 
      filename="elk_projects"+"_"+str(datetime.now())+".csv"
      elk_cdc.to_csv(filename,index=False)
      #counting the number of projects
      total_projects=pd.read_sql("SELECT COUNT(*) AS total_projects FROM elk_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
      project_count=str(int(total_projects.iloc[0]))
      #customization of notification that has to be sent to slack channel
      notification="Update :"+" "+project_count+" "+"infrastructure projects indentified in Elk Grove"
      try:
          client.chat_postMessage(channel="california_infra_projects",text=notification,username="infra_alerts")
          client.files_upload_v2(channel="C07AH45FZDZ",file=filename)
      except SlackApiError as e:
          print(f"Error in sending message:{e}")

  def close_driver(self):
    self.driver.quit()

if __name__ == "__main__":
  headers = ['project_code','title','description','applicant','status','project_materials']
  elkgrover_extractor = ElkGroveDataExtractor()
  elkgrover_data = elkgrover_extractor.extract_data()
  elkgrover_extractor.save_to_csv(elkgrover_data,headers)
  modified_data = elkgrover_extractor.cleanup_data(elkgrover_data)
  modified_data.to_csv('modified_elk.csv',index=False,na_rep='NULL')
  elkgrover_extractor.close_driver()
  elkgrover_extractor.insert_data()
  elkgrover_extractor.send_alerts()

In [ ]:
#eracting arcata data

class ArcataDataExtractor:
  def __init__(self):
      self.options = webdriver.FirefoxOptions()
      self.options.add_argument("--headless")
      self.driver = webdriver.Firefox(options=self.options)
      load_dotenv('/root/Desktop/infra/cred.env')
      self.arcata_url=os.environ.get('ARCATA_URL')
      db_host=os.environ.get('DB_HOST')
      db_user=os.environ.get('DB_USER')
      db_password=os.environ.get('DB_PASSWORD')
      self.mydb=mysql.connector.connect(
          host=db_host,
          user=db_user,
          password=db_password
        )

  def extract_data(self):
      # Create an empty dataframe with the desired columns
      arcata_df = pd.DataFrame(columns=['original_id', 'aug_id', 'country_name', 'country_code', 'map_coordinates', 'url', 'region_name', 'region_code', 'title', 'description', 'status', 'stages', 'date', 'procurement_method', 'budget', 'currency', 'buyer', 'sector', 'subsector','location'])
      # get the webpage
      self.driver.execute_script(f"location.href='{self.arcata_url}';")
      # self.driver.implicitly_wait(10)
      headline=WebDriverWait(self.driver,30).until(
          EC.presence_of_element_located((By.CSS_SELECTOR,"#versionHeadLine"))
      )
      # find the table with CSS selector
      arcata_table = self.driver.find_element('css selector', "div[class='widgetBody'] table")
      # Get all the rows from the table
      arcata_rows = arcata_table.find_elements('css selector', "tr")
      # Iterate through the row and accumulate title, description, status
      for i in range(1, len(arcata_rows)):
          # Scroll down the page by 100 pixels
          self.driver.execute_script("window.scrollBy(0,100)")
          # extract the project title
          project_title = self.driver.find_element('css selector', f'tbody tr:nth-child({i}) td:nth-child(1) strong:nth-child(1)')
          # extract the description
          project_description = self.driver.find_element('css selector', f'tbody tr:nth-child({i}) td:nth-child(2)')
          # extract the status
          project_status = self.driver.find_element('css selector', f'tbody tr:nth-child({i}) td:nth-child(3)')
          # regex pattern to extract budget
          budget_pattern = r'\$(\d+(?:,\d{3})*(?:\.\d+)?)(?:\s*(million|billion|thousand))?'
          # Search for the pattern in the text
          bud_match = re.search(budget_pattern, project_description.text)
          if bud_match:
              project_budget = bud_match.group(0)
          else:
              project_budget = "null"
          # regex pattern to extract buyer
          buyer_pattern = r'contracted with\s+([^\d.,;]+)\b'
          # Find matches in the text
          buy_match = re.search(buyer_pattern, project_description.text)
          # Print the matches
          if buy_match:
              project_buyer = buy_match.group(0)
          else:
              project_buyer = "null"
          arcata_df = arcata_df._append({"title": project_title.text, "description": project_description.text,
                                          "status": project_status.text, "budget": project_budget,"url":self.arcata_url,
                                          "buyer": project_buyer}, ignore_index=True)
      # Replace empty values with NaN
      arcata_df = arcata_df.replace(r'^\s*$', np.nan, regex=True)
      #assiging uuid for each row
      arcata_df['aug_id'] = [str(uuid.uuid4()) for _ in range(len(arcata_df))]
      return arcata_df
      
  def insert_data(self):
      mycursor = self.mydb.cursor()
      with open('modified_arcata.csv') as file:
          rows=csv.reader(file)
          next(rows)
          val=[tuple(row) for row in rows]
      query="INSERT INTO arcata_source (original_id,aug_id,country_name,country_code,map_coordinates,url,region_name,region_code,title,description,status,stages,published_date,procurement_method,budget,currency,buyer,sector,subsector,location)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
      mycursor.execute("USE infraDB")
      mycursor.executemany(query,val)
      self.mydb.commit()

  def send_alerts(self):
      slack_token=os.environ.get('SLACK_BOT_TOKEN')
      #intialize webclient instance with OAtuh token
      client=WebClient(token=slack_token)
      #fetch the updated records from the cdc
      arcata_cdc=pd.read_sql("SELECT*FROM arcata_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
      #create a csv filename which accumulates updated cdc records 
      filename="arcata_projects"+"_"+str(datetime.now())+".csv"
      arcata_cdc.to_csv(filename,index=False)
      #counting the number of projects
      total_projects=pd.read_sql("SELECT COUNT(*) AS total_projects FROM  arcata_cdc WHERE DATE(last_modified_dt)=DATE(NOW())",self.mydb)
      project_count=str(int(total_projects.iloc[0]))
      #customization of notification that has to be sent to slack channel
      notification="Update :"+" "+project_count+" "+"infrastructure projects indentified in Arcata"
      try:
          client.chat_postMessage(channel="california_infra_projects",text=notification,username="infra_alerts")
          client.files_upload_v2(channel="C07AH45FZDZ",file=filename)
      except SlackApiError as e:
          print(f"Error in sending message:{e}")

  def close_driver(self):
      self.driver.quit()

if __name__ == "__main__":
  arcata_extractor = ArcataDataExtractor()
  arcata_data = arcata_extractor.extract_data()
  arcata_data.to_csv('modified_arcata.csv', index=False, na_rep='NULL')
  arcata_extractor.insert_data()
  arcata_extractor.close_driver()
  arcata_extractor.send_alerts()